In [98]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import xml.etree.ElementTree

def plot_img_inline(img):
    plt.figure()
    plt.imshow(np.asarray(img))
    plt.show()

char_count = dict()

def get_unique_img_name(target_path, index_name, char_name):
    idx = 0
    image_filename = '.'.join([index_name, char_name, str(idx), 'jpg'])
    full_path = join(target_path, image_filename)
    
    while path.exists(full_path):
        idx = idx + 1
        image_filename = '.'.join([index_name, char_name, str(idx), 'jpg'])
        full_path = join(target_path, image_filename)
    return full_path

def print_img(img, base_path, index_name, char_name):
    target_path = join(base_path, char_name)
    if not path.exists(target_path):
        makedirs(target_path)

    full_path = get_unique_img_name(target_path, index_name, char_name)
    img.save(full_path)
    
    if char_name not in char_count:
        char_count[char_name] = 1
    else:
        char_count[char_name] = char_count[char_name] + 1
    
def crop_character_images(base_path, index_name, output_path='char_data'):
    image_filename = '.'.join([index_name, 'jpg'])
    xml_filename = '.'.join([index_name, 'xml'])
    if path.exists(join(base_path, xml_filename)) is False:
        xml_filename = index_name
    im = Image.open(join(base_path, image_filename))
    
#     plot_img_inline(im)
    e = xml.etree.ElementTree.parse(join(base_path, xml_filename)).getroot()

    for image in e:
        for words in image:
            if words.tag == 'words':
                for word in words:
                    for char in word:
                        if len(char.attrib['char']) != 1:
                            continue
                        if char.attrib['char'] == '.':
                            char.attrib['char'] = 'dot'
                        box = (int(char.attrib['x']), int(char.attrib['y']), int(char.attrib['x']) + int(char.attrib['width']), int(char.attrib['y']) + int(char.attrib['height']))
                        region = im.crop(box)
#                         plot_img_inline(region)
                        print_img(region, output_path, index_name, char.attrib['char'])

In [100]:
from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

img_base_path = 'kaist_dataset'

'''
result = {
    path1: {
        path2: {
            
        }
    }
}
'''

fail_filenames = []
def get_recursive_dirlist(base_path):
#     print('###base_path', base_path)
    sub_dirnames = [f for f in listdir(base_path) if isdir(join(base_path, f))]
    for dirname in sub_dirnames:
        next_base_path = join(base_path, dirname)
        get_recursive_dirlist(next_base_path)
        data_filenames = [path.splitext(f)[0] for f in listdir(next_base_path) if isfile(join(next_base_path, f)) and f.lower().endswith('.jpg')]
        
        for x in data_filenames:
            try:
                crop_character_images(next_base_path, x)
            except:
                print('###EXCEPTION', next_base_path, x)
                fail_filenames.append((next_base_path, x))
#         list(map(lambda x: crop_character_images(next_base_path, x), data_filenames))

get_recursive_dirlist(img_base_path)

###EXCEPTION kaist_dataset/English/Digital_Camera/(E.S)F-others 4
###EXCEPTION kaist_dataset/Korean/Digital_Camera/G-others 080116-0079
###EXCEPTION kaist_dataset/Korean/Digital_Camera/indoor1 080119-0001
###EXCEPTION kaist_dataset/Korean/Digital_Camera/outdoor1 99
###EXCEPTION kaist_dataset/Korean/Digital_Camera/outdoor2 080119-0031
###EXCEPTION kaist_dataset/Korean/Digital_Camera/outdoor2 2007-12-08-day-122
###EXCEPTION kaist_dataset/Mixed/Digital_Camera/(C.S)C-outdoor1 063
###EXCEPTION kaist_dataset/Mixed/Digital_Camera/(C.S)C-outdoor1 080119-0003
